In [1]:
from qaoa import *




In [2]:
# Example 1: Creating an Ising Hamiltonian and Finding Fisher Information Matrix
# Set a random seed for reproducibility
np.random.seed(42)

# Define the number of qubits and connectivity type ("open" or "closed")
n_qubits = 4
connectivity = "closed"

# Create an Ising Hamiltonian
ising_hamiltonian = H_zz_Ising(n_qubits, connectivity)

depth = 1
# Initialize QAOA with a specific depth and the Ising Hamiltonian
qaoa = QAOA(depth=depth, H=ising_hamiltonian)

# Define a set of parameters
angles = [0.5, 0.2]*depth  # Adjust these parameters as needed

# Calculate the Fisher Information Matrix
qfi_matrix = qaoa.qaoa_qfi_matrix(angles, state_ini=plus_state(n_qubits)) #|+>

print("Fisher Information Matrix:")
with np.printoptions(precision=3, suppress=True):
    print(qfi_matrix)


Fisher Information Matrix:
[[16.     0.   ]
 [ 0.    29.273]]


In [3]:
# Example 2: Creating a max k-SAT Hamiltonian and Optimizing it with QAOA
# Set a random seed for reproducibility
np.random.seed(42)
# Define the number of variables (n), clauses (k), and alpha for k-SAT Hamiltonian
n = 5
k = 3
alpha = 2.0

# Create a k-SAT Hamiltonian
k_sat_hamiltonian = H_sat(n, k, alpha)

depth = 2
# Initialize QAOA with a specific depth and the k-SAT Hamiltonian
qaoa = QAOA(depth=depth, H=k_sat_hamiltonian)


# Run the QAOA optimization using the L-BFGS-B method
qaoa.run()

# Print the optimization results
with np.printoptions(precision=3, suppress=True):
    print("QAOA Parameters:", qaoa.opt_angles)
    print("Optimized QAOA Energy:", qaoa.q_energy)


QAOA Parameters: [1.261 3.682 1.142 1.765]
Optimized QAOA Energy: 0.5039655121265381


In [4]:
# Example 3: Solving the Max Cut Problem
# Set a random seed for reproducibility
np.random.seed(42)
# Define the graph adjacency matrix (replace this with your specific graph)
graph = [
    [0, 1, 1, 0],
    [1, 0, 0, 1],
    [1, 0, 0, 1],
    [0, 1, 1, 0]
]

# Number of qubits, which is the same as the number of nodes in the graph will be calculated inside the function
# Convert the graph into the corresponding Ising Hamiltonian
max_cut_hamiltonian = Graph_to_Hamiltonian(graph)

depth = 4
# Now you can use your QAOA solver to find the maximum cut
qaoa = QAOA(depth=depth, H=max_cut_hamiltonian)
qaoa.run()

# Extract the results
max_cut_solution = qaoa.f_state  # The quantum state
max_cut_value = qaoa.q_energy  # The value of the maximum cut

# Output the results
with np.printoptions(precision=3, suppress=True):
    print("Max Cut Solution (quantum state):\n", max_cut_solution)
    print("Max Cut Value:", max_cut_value)

Max Cut Solution (quantum state):
 [-0.   -0.j     0.   +0.j     0.   +0.j     0.   -0.j     0.   +0.j
  0.   -0.j     0.176-0.685j  0.   +0.j     0.   +0.j     0.176-0.685j
  0.   -0.j     0.   +0.j     0.   -0.j     0.   +0.j     0.   +0.j
 -0.   -0.j   ]
Max Cut Value: -3.9999999999998725


In [5]:
# Example 4: Solving the Max k-SAT with PROTES method 
# Set a random seed for reproducibility
np.random.seed(42)

# Define the number of variables (n), clauses (k), and alpha for k-SAT Hamiltonian
n = 4
k = 2
alpha = 1.0

# Create a k-SAT Hamiltonian
k_sat_hamiltonian = H_sat(n, k, alpha)

depth = 2
# Initialize QAOA with a specific depth and the k-SAT Hamiltonian
qaoa = QAOA(depth=depth, H=k_sat_hamiltonian)
#run the PROTES method 
qaoa.run_PROTES()

# Output the results
with np.printoptions(precision=3, suppress=True):
    print("Max k-SAT solution (quantum state):\n",  qaoa.f_state)
    print("Max k-SAT energy:", qaoa.q_energy)

# for the PROTES only method there is a special value protes_log stored in qaoa
# Print additional information from PROTES log
protes_log = qaoa.protes_log
print("Time to execute:", protes_log['t'])
print("Minimum Candidates:")
for i, (i_opt, y_opt) in enumerate(zip(protes_log['i_opt_list'], protes_log['y_opt_list']), 1):
    angles = 360 * i_opt/100  # if you change linspace params inside protes then you have to change it here for the right plot
    print(f" Angles: {angles}°, Value: {y_opt}")

Max k-SAT solution (quantum state):
 [ 0.475+0.094j -0.069-0.037j -0.01 -0.082j -0.031+0.036j  0.475+0.094j
 -0.069-0.037j -0.01 -0.082j -0.031+0.036j -0.031+0.036j -0.069-0.037j
 -0.01 -0.082j  0.475+0.094j -0.031+0.036j -0.069-0.037j -0.01 -0.082j
  0.475+0.094j]
Max k-SAT energy: 0.08777282
Time to execute: 5.271177499998885
Minimum Candidates:
 Angles: [ 72.   10.8  90.  244.8]°, Value: 0.3070797622203827
 Angles: [108.  273.6 212.4 327.6]°, Value: 0.14903999865055084
 Angles: [ 79.2 126.  324.  154.8]°, Value: 0.11032555997371674
 Angles: [ 75.6 201.6 158.4 205.2]°, Value: 0.08777281641960144


In [17]:
# Example 5: Calculate the Average Expectation with Noise
# Set a random seed for reproducibility
np.random.seed(42)
# Define the number of qubits and connectivity type ("open" or "closed")
n_qubits = 4
connectivity = "closed"

# Define the depth of the QAOA circuit
depth = 2

# Create an Ising Hamiltonian
ising_hamiltonian = H_zz_Ising(n_qubits, connectivity)

# Initialize QAOA with a specific depth and the Ising Hamiltonian
qaoa = QAOA(depth=depth, H=ising_hamiltonian)


# Find optimum angles
qaoa = QAOA(depth=depth, H=ising_hamiltonian)
qaoa.run()
angles = qaoa.opt_angles  # Adjust these parameters as needed
noiseless_energy = qaoa.q_energy

# Create Kraus operators and probabilities for the depolarization channel
p_depolarization = 0.01  # Adjust the depolarization probability
depolarization_kraus_ops, depolarization_probabilities = create_depolarization_kraus(p_depolarization)


# Apply the QAOA ansatz with depolarization noise
energy_with_noise = qaoa.expectation_noise(angles, noise_prob=depolarization_probabilities, kraus_ops=depolarization_kraus_ops,num_samples=100)

# Print the state vector with depolarization noise
# Output the results
with np.printoptions(precision=3, suppress=True):
    print("Average Expectation with Depolarization Noise:{:.2f}".format(energy_with_noise))
    print("Expectation without Noise:{:.2f}".format(noiseless_energy))
    print("Overlap between solution and state with noise:{:.2f}".format(qaoa.olap))
    print("State with noise:", qaoa.f_state)

Average Expectation with Depolarization Noise:-0.75
Expectation without Noise:-2.00
Overlap between solution and state with noise:0.53
State with noise: [-0.125-0.j     0.   -0.125j  0.   -0.125j  0.125-0.25j   0.   -0.125j
 -0.125-0.5j    0.125-0.25j   0.   -0.125j  0.   -0.125j  0.125-0.25j
 -0.125-0.5j    0.   -0.125j  0.125-0.25j   0.   -0.125j  0.   -0.125j
 -0.125-0.j   ]
